In [15]:
import os
import fnmatch

def get_directory_structure(path, max_file_size=50000, exclude_patterns=None):
    """
    Lấy cấu trúc thư mục và nội dung file
    """
    if exclude_patterns is None:
        exclude_patterns = [
            '*.pyc', '__pycache__', '.git', 'node_modules', 
            '*.jpg', '*.png', '*.gif', '*.pdf', '*.zip',
            '.env', '*.log', 'venv', '.venv'
        ]
    
    result = []
    
    def should_exclude(file_path):
        for pattern in exclude_patterns:
            if fnmatch.fnmatch(file_path, pattern) or pattern in file_path:
                return True
        return False
    
    def process_directory(current_path, prefix=""):
        try:
            items = sorted(os.listdir(current_path))
            for item in items:
                item_path = os.path.join(current_path, item)
                
                if should_exclude(item_path):
                    continue
                
                if os.path.isdir(item_path):
                    result.append(f"{prefix}📁 {item}/")
                    process_directory(item_path, prefix + "  ")
                else:
                    result.append(f"{prefix}📄 {item}")
                    
                    # Đọc nội dung file nếu không quá lớn
                    if os.path.getsize(item_path) < max_file_size:
                        try:
                            with open(item_path, 'r', encoding='utf-8') as f:
                                content = f.read()
                                if content.strip():
                                    result.append(f"{prefix}  === Nội dung {item} ===")
                                    result.append(content[:2000] + "..." if len(content) > 2000 else content)
                                    result.append(f"{prefix}  === Kết thúc {item} ===")
                        except:
                            result.append(f"{prefix}  (Không thể đọc file)")
                    else:
                        result.append(f"{prefix}  (File quá lớn)")
        except PermissionError:
            result.append(f"{prefix}❌ Không có quyền truy cập")
    
    result.append(f"📂 Cấu trúc thư mục: {os.path.basename(path)}")
    result.append("=" * 50)
    process_directory(path)
    
    return "\n".join(result)

# Sử dụng
project_path = "."  # Thay đổi đường dẫn tại đây
output = get_directory_structure(project_path)

# Lưu vào file
with open("project_structure.txt", "w", encoding="utf-8") as f:
    f.write(output)

print("Đã lưu cấu trúc vào file project_structure.txt")
print("\nPreview:")
print(output[:1000] + "..." if len(output) > 1000 else output)


Đã lưu cấu trúc vào file project_structure.txt

Preview:
📂 Cấu trúc thư mục: .
📄 Dockerfile
  === Nội dung Dockerfile ===
FROM node:18-alpine

WORKDIR /app

# Copy package files
COPY package*.json ./

# Install dependencies
RUN npm install --only=production

# Copy application code
COPY . .

# Create non-root user
RUN addgroup -g 1001 -S nodejs
RUN adduser -S nodejs -u 1001

# Change ownership of the app directory
RUN chown -R nodejs:nodejs /app

# Switch to non-root user
USER nodejs

# Expose port
EXPOSE 8080

# Start the application
CMD ["npm", "start"]

  === Kết thúc Dockerfile ===
📄 Getcode.ipynb
  === Nội dung Getcode.ipynb ===
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "68a6e605",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "3f21e831",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Đã lưu cấu trúc vào

In [11]:
# import requests
# import json

# class LarkBaseClient:
#     def __init__(self, app_id, app_secret):
#         self.app_id = app_id
#         self.app_secret = app_secret
#         self.access_token = None
#         self.base_url = "https://open.larksuite.com/open-apis"
    
#     def get_access_token(self):
#         """Lấy tenant access token"""
#         url = f"{self.base_url}/auth/v3/tenant_access_token/internal"
        
#         payload = {
#             "app_id": self.app_id,
#             "app_secret": self.app_secret
#         }
        
#         response = requests.post(url, json=payload)
        
#         if response.status_code == 200:
#             data = response.json()
#             self.access_token = data.get('tenant_access_token')
#             return True
#         else:
#             print(f"Lỗi lấy token: {response.text}")
#             return False
    
#     def get_record(self, app_token, table_id, record_id):
#         """Lấy dữ liệu record"""
#         if not self.access_token:
#             if not self.get_access_token():
#                 return None
        
#         url = f"{self.base_url}/bitable/v1/apps/{app_token}/tables/{table_id}/records/{record_id}"
        
#         headers = {
#             "Authorization": f"Bearer {self.access_token}",
#             "Content-Type": "application/json"
#         }
        
#         response = requests.get(url, headers=headers)
        
#         if response.status_code == 200:
#             return response.json()
#         else:
#             print(f"Lỗi API: {response.status_code} - {response.text}")
#             return None

# # Sử dụng
# if __name__ == "__main__":
#     # Khởi tạo client
#     client = LarkBaseClient("cli_a7fab27260385010", "Zg4MVcFfiOu0g09voTcpfd4WGDpA0Ly5")
    
#     # Lấy dữ liệu
#     record_data = client.get_record(
#         app_token="Ey3EbVD9vacAHvs8cVvlHxkKg2r",
#         table_id="tblX52T3kxH2DN00", 
#         record_id="recuRbIPbr0KeV"
#     )
    
#     if record_data:
#         print(json.dumps(record_data, indent=2, ensure_ascii=False))


In [1]:
import os
import re
import json
from pathlib import Path

def get_project_structure(root_path='.', exclude_dirs=None, exclude_files=None):
    """
    Lấy cấu trúc thư mục của dự án
    """
    if exclude_dirs is None:
        exclude_dirs = {'.git', '__pycache__', '.venv', 'venv', 'node_modules', '.pytest_cache'}
    
    if exclude_files is None:
        exclude_files = {'.gitignore', '.pyc', '.pyo', '.pyd', '.so', '.egg-info'}
    
    structure = []
    
    def should_exclude(path):
        name = os.path.basename(path)
        return any(exc in name for exc in exclude_files) or name.startswith('.')
    
    def build_tree(path, prefix="", is_last=True):
        if os.path.basename(path) in exclude_dirs:
            return
        
        items = []
        try:
            for item in sorted(os.listdir(path)):
                item_path = os.path.join(path, item)
                if not should_exclude(item_path):
                    items.append((item, item_path))
        except PermissionError:
            return
        
        for i, (item, item_path) in enumerate(items):
            is_last_item = i == len(items) - 1
            
            if os.path.isdir(item_path):
                connector = "└── " if is_last_item else "├── "
                structure.append(f"{prefix}{connector}{item}/")
                
                next_prefix = prefix + ("    " if is_last_item else "│   ")
                build_tree(item_path, next_prefix, is_last_item)
            else:
                connector = "└── " if is_last_item else "├── "
                structure.append(f"{prefix}{connector}{item}")
    
    project_name = os.path.basename(os.path.abspath(root_path))
    structure.append(f"{project_name}/")
    build_tree(root_path)
    
    return structure

def analyze_javascript_file(file_path):
    """
    Phân tích file JavaScript để lấy ra functions, classes, exports
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        functions = []
        classes = []
        exports = []
        
        # Tìm function declarations
        func_pattern = r'function\s+([a-zA-Z_$][a-zA-Z0-9_$]*)\s*\([^)]*\)'
        func_matches = re.finditer(func_pattern, content)
        for match in func_matches:
            line_num = content[:match.start()].count('\n') + 1
            functions.append({
                'name': match.group(1),
                'line': line_num,
                'type': 'function declaration'
            })
        
        # Tìm arrow functions
        arrow_pattern = r'const\s+([a-zA-Z_$][a-zA-Z0-9_$]*)\s*=\s*\([^)]*\)\s*=>'
        arrow_matches = re.finditer(arrow_pattern, content)
        for match in arrow_matches:
            line_num = content[:match.start()].count('\n') + 1
            functions.append({
                'name': match.group(1),
                'line': line_num,
                'type': 'arrow function'
            })
        
        # Tìm method trong object
        method_pattern = r'([a-zA-Z_$][a-zA-Z0-9_$]*)\s*:\s*function\s*\([^)]*\)'
        method_matches = re.finditer(method_pattern, content)
        for match in method_matches:
            line_num = content[:match.start()].count('\n') + 1
            functions.append({
                'name': match.group(1),
                'line': line_num,
                'type': 'method'
            })
        
        # Tìm async function
        async_pattern = r'async\s+function\s+([a-zA-Z_$][a-zA-Z0-9_$]*)\s*\([^)]*\)'
        async_matches = re.finditer(async_pattern, content)
        for match in async_matches:
            line_num = content[:match.start()].count('\n') + 1
            functions.append({
                'name': match.group(1),
                'line': line_num,
                'type': 'async function'
            })
        
        # Tìm class declarations
        class_pattern = r'class\s+([a-zA-Z_$][a-zA-Z0-9_$]*)'
        class_matches = re.finditer(class_pattern, content)
        for match in class_matches:
            line_num = content[:match.start()].count('\n') + 1
            classes.append({
                'name': match.group(1),
                'line': line_num
            })
        
        # Tìm exports
        export_patterns = [
            r'module\.exports\s*=\s*([a-zA-Z_$][a-zA-Z0-9_$]*)',
            r'exports\.([a-zA-Z_$][a-zA-Z0-9_$]*)',
            r'export\s+(?:default\s+)?([a-zA-Z_$][a-zA-Z0-9_$]*)',
            r'export\s*\{([^}]+)\}'
        ]
        
        for pattern in export_patterns:
            matches = re.finditer(pattern, content)
            for match in matches:
                line_num = content[:match.start()].count('\n') + 1
                exports.append({
                    'name': match.group(1),
                    'line': line_num
                })
        
        return functions, classes, exports
    
    except Exception as e:
        print(f"Lỗi khi phân tích file {file_path}: {e}")
        return [], [], []

def analyze_python_file(file_path):
    """
    Phân tích file Python (giữ nguyên từ code cũ)
    """
    try:
        import ast
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        tree = ast.parse(content)
        
        classes = []
        functions = []
        
        for node in ast.walk(tree):
            if isinstance(node, ast.ClassDef):
                methods = []
                for item in node.body:
                    if isinstance(item, ast.FunctionDef):
                        params = []
                        for arg in item.args.args:
                            params.append(arg.arg)
                        method_info = {
                            'name': item.name,
                            'params': params,
                            'line': item.lineno
                        }
                        methods.append(method_info)
                
                class_info = {
                    'name': node.name,
                    'line': node.lineno,
                    'methods': methods
                }
                classes.append(class_info)
            
            elif isinstance(node, ast.FunctionDef):
                params = []
                for arg in node.args.args:
                    params.append(arg.arg)
                
                function_info = {
                    'name': node.name,
                    'params': params,
                    'line': node.lineno
                }
                functions.append(function_info)
        
        return classes, functions
    
    except Exception as e:
        print(f"Lỗi khi phân tích file {file_path}: {e}")
        return [], []

def get_all_code_files(root_path='.'):
    """
    Lấy tất cả file code trong dự án (Python, JavaScript, TypeScript, etc.)
    """
    code_files = []
    exclude_dirs = {'.git', '__pycache__', '.venv', 'venv', 'node_modules', '.pytest_cache'}
    
    # Các extension file code cần phân tích
    code_extensions = {'.py', '.js', '.ts', '.jsx', '.tsx', '.vue', '.php', '.java', '.c', '.cpp', '.cs'}
    
    for root, dirs, files in os.walk(root_path):
        dirs[:] = [d for d in dirs if d not in exclude_dirs]
        
        for file in files:
            file_ext = os.path.splitext(file)[1].lower()
            if file_ext in code_extensions:
                file_path = os.path.join(root, file)
                code_files.append(file_path)
    
    return code_files

def generate_report(output_file='project_analysis.txt'):
    """
    Tạo báo cáo tổng hợp và xuất ra file txt
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("=" * 80 + "\n")
        f.write("              PHÂN TÍCH CẤU TRÚC DỰ ÁN\n")
        f.write("=" * 80 + "\n\n")
        
        # 1. Cấu trúc thư mục
        f.write("1. CẤU TRÚC THƯ MỤC:\n")
        f.write("-" * 40 + "\n")
        
        structure = get_project_structure()
        for line in structure:
            f.write(line + "\n")
        
        f.write("\n")
        
        # 2. Phân tích từng file code
        f.write("2. PHÂN TÍCH CÁC FILE CODE:\n")
        f.write("-" * 40 + "\n")
        
        code_files = get_all_code_files()
        
        if not code_files:
            f.write("Không tìm thấy file code nào trong dự án.\n")
        else:
            python_files = [f for f in code_files if f.endswith('.py')]
            js_files = [f for f in code_files if f.endswith(('.js', '.jsx', '.ts', '.tsx'))]
            other_files = [f for f in code_files if f not in python_files and f not in js_files]
            
            # Phân tích file Python
            if python_files:
                f.write("\n🐍 PYTHON FILES:\n")
                f.write("=" * 50 + "\n")
                
                for file_path in python_files:
                    f.write(f"\n📁 File: {file_path}\n")
                    f.write("-" * 30 + "\n")
                    
                    classes, functions = analyze_python_file(file_path)
                    
                    if classes:
                        f.write("🏛️  CLASSES:\n")
                        for cls in classes:
                            f.write(f"   • {cls['name']} (dòng {cls['line']})\n")
                            if cls['methods']:
                                for method in cls['methods']:
                                    params_str = ", ".join(method['params'])
                                    f.write(f"     - {method['name']}({params_str}) (dòng {method['line']})\n")
                    
                    if functions:
                        f.write("🔧 FUNCTIONS:\n")
                        for func in functions:
                            params_str = ", ".join(func['params'])
                            f.write(f"   • {func['name']}({params_str}) (dòng {func['line']})\n")
                    
                    if not classes and not functions:
                        f.write("   Không có class hoặc function nào được tìm thấy.\n")
            
            # Phân tích file JavaScript/TypeScript
            if js_files:
                f.write("\n🟨 JAVASCRIPT/TYPESCRIPT FILES:\n")
                f.write("=" * 50 + "\n")
                
                for file_path in js_files:
                    f.write(f"\n📁 File: {file_path}\n")
                    f.write("-" * 30 + "\n")
                    
                    functions, classes, exports = analyze_javascript_file(file_path)
                    
                    if classes:
                        f.write("🏛️  CLASSES:\n")
                        for cls in classes:
                            f.write(f"   • {cls['name']} (dòng {cls['line']})\n")
                    
                    if functions:
                        f.write("🔧 FUNCTIONS:\n")
                        for func in functions:
                            f.write(f"   • {func['name']} (dòng {func['line']}) - {func['type']}\n")
                    
                    if exports:
                        f.write("📤 EXPORTS:\n")
                        for exp in exports:
                            f.write(f"   • {exp['name']} (dòng {exp['line']})\n")
                    
                    if not classes and not functions and not exports:
                        f.write("   Không có class, function hoặc export nào được tìm thấy.\n")
            
            # Liệt kê các file khác
            if other_files:
                f.write("\n📄 OTHER CODE FILES:\n")
                f.write("=" * 50 + "\n")
                for file_path in other_files:
                    f.write(f"   • {file_path}\n")
        
        # 3. Thống kê tổng quan
        f.write("\n3. THỐNG KÊ TỔNG QUAN:\n")
        f.write("-" * 40 + "\n")
        
        total_files = len(code_files)
        python_count = len([f for f in code_files if f.endswith('.py')])
        js_count = len([f for f in code_files if f.endswith(('.js', '.jsx', '.ts', '.tsx'))])
        other_count = total_files - python_count - js_count
        
        f.write(f"📊 Tổng số file code: {total_files}\n")
        f.write(f"📊 File Python: {python_count}\n")
        f.write(f"📊 File JavaScript/TypeScript: {js_count}\n")
        f.write(f"📊 File khác: {other_count}\n")
        
        f.write("\n" + "=" * 80 + "\n")
        f.write("Báo cáo được tạo thành công!\n")
        f.write("=" * 80 + "\n")

# Chạy chương trình
if __name__ == "__main__":
    print("Đang phân tích cấu trúc dự án...")
    generate_report()
    print("✅ Hoàn thành! Kiểm tra file 'project_analysis.txt'")


Đang phân tích cấu trúc dự án...
✅ Hoàn thành! Kiểm tra file 'project_analysis.txt'


In [2]:
import os
import re
import json
from pathlib import Path


def get_project_structure(root_path='.', exclude_dirs=None, exclude_files=None):
    """
    Lấy cấu trúc thư mục của dự án
    """
    if exclude_dirs is None:
        exclude_dirs = {'.git', '__pycache__', '.venv', 'venv', 'node_modules', '.pytest_cache'}
    
    if exclude_files is None:
        exclude_files = {'.gitignore', '.pyc', '.pyo', '.pyd', '.so', '.egg-info'}
    
    structure = []
    
    def should_exclude(path):
        name = os.path.basename(path)
        return any(exc in name for exc in exclude_files) or name.startswith('.')
    
    def build_tree(path, prefix="", is_last=True):
        if os.path.basename(path) in exclude_dirs:
            return
        
        items = []
        try:
            for item in sorted(os.listdir(path)):
                item_path = os.path.join(path, item)
                if not should_exclude(item_path):
                    items.append((item, item_path))
        except PermissionError:
            return
        
        for i, (item, item_path) in enumerate(items):
            is_last_item = i == len(items) - 1
            
            if os.path.isdir(item_path):
                connector = "└── " if is_last_item else "├── "
                structure.append(f"{prefix}{connector}{item}/")
                
                next_prefix = prefix + ("    " if is_last_item else "│   ")
                build_tree(item_path, next_prefix, is_last_item)
            else:
                connector = "└── " if is_last_item else "├── "
                structure.append(f"{prefix}{connector}{item}")
    
    project_name = os.path.basename(os.path.abspath(root_path))
    structure.append(f"{project_name}/")
    build_tree(root_path)
    
    return structure


def get_file_content(file_path):
    """
    Lấy nội dung đầy đủ của file
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except UnicodeDecodeError:
        try:
            with open(file_path, 'r', encoding='latin-1') as file:
                content = file.read()
            return content
        except Exception as e:
            return f"[Lỗi khi đọc file: {e}]"
    except Exception as e:
        return f"[Lỗi khi đọc file: {e}]"


def analyze_javascript_file(file_path):
    """
    Phân tích file JavaScript để lấy ra functions, classes, exports
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        functions = []
        classes = []
        exports = []
        
        # Tìm function declarations
        func_pattern = r'function\s+([a-zA-Z_$][a-zA-Z0-9_$]*)\s*\([^)]*\)'
        func_matches = re.finditer(func_pattern, content)
        for match in func_matches:
            line_num = content[:match.start()].count('\n') + 1
            functions.append({
                'name': match.group(1),
                'line': line_num,
                'type': 'function declaration'
            })
        
        # Tìm arrow functions
        arrow_pattern = r'const\s+([a-zA-Z_$][a-zA-Z0-9_$]*)\s*=\s*\([^)]*\)\s*=>'
        arrow_matches = re.finditer(arrow_pattern, content)
        for match in arrow_matches:
            line_num = content[:match.start()].count('\n') + 1
            functions.append({
                'name': match.group(1),
                'line': line_num,
                'type': 'arrow function'
            })
        
        # Tìm method trong object
        method_pattern = r'([a-zA-Z_$][a-zA-Z0-9_$]*)\s*:\s*function\s*\([^)]*\)'
        method_matches = re.finditer(method_pattern, content)
        for match in method_matches:
            line_num = content[:match.start()].count('\n') + 1
            functions.append({
                'name': match.group(1),
                'line': line_num,
                'type': 'method'
            })
        
        # Tìm async function
        async_pattern = r'async\s+function\s+([a-zA-Z_$][a-zA-Z0-9_$]*)\s*\([^)]*\)'
        async_matches = re.finditer(async_pattern, content)
        for match in async_matches:
            line_num = content[:match.start()].count('\n') + 1
            functions.append({
                'name': match.group(1),
                'line': line_num,
                'type': 'async function'
            })
        
        # Tìm class declarations
        class_pattern = r'class\s+([a-zA-Z_$][a-zA-Z0-9_$]*)'
        class_matches = re.finditer(class_pattern, content)
        for match in class_matches:
            line_num = content[:match.start()].count('\n') + 1
            classes.append({
                'name': match.group(1),
                'line': line_num
            })
        
        # Tìm exports
        export_patterns = [
            r'module\.exports\s*=\s*([a-zA-Z_$][a-zA-Z0-9_$]*)',
            r'exports\.([a-zA-Z_$][a-zA-Z0-9_$]*)',
            r'export\s+(?:default\s+)?([a-zA-Z_$][a-zA-Z0-9_$]*)',
            r'export\s*\{([^}]+)\}'
        ]
        
        for pattern in export_patterns:
            matches = re.finditer(pattern, content)
            for match in matches:
                line_num = content[:match.start()].count('\n') + 1
                exports.append({
                    'name': match.group(1),
                    'line': line_num
                })
        
        return functions, classes, exports
    
    except Exception as e:
        print(f"Lỗi khi phân tích file {file_path}: {e}")
        return [], [], []


def analyze_python_file(file_path):
    """
    Phân tích file Python
    """
    try:
        import ast
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        tree = ast.parse(content)
        
        classes = []
        functions = []
        
        for node in ast.walk(tree):
            if isinstance(node, ast.ClassDef):
                methods = []
                for item in node.body:
                    if isinstance(item, ast.FunctionDef):
                        params = []
                        for arg in item.args.args:
                            params.append(arg.arg)
                        method_info = {
                            'name': item.name,
                            'params': params,
                            'line': item.lineno
                        }
                        methods.append(method_info)
                
                class_info = {
                    'name': node.name,
                    'line': node.lineno,
                    'methods': methods
                }
                classes.append(class_info)
            
            elif isinstance(node, ast.FunctionDef):
                params = []
                for arg in node.args.args:
                    params.append(arg.arg)
                
                function_info = {
                    'name': node.name,
                    'params': params,
                    'line': node.lineno
                }
                functions.append(function_info)
        
        return classes, functions
    
    except Exception as e:
        print(f"Lỗi khi phân tích file {file_path}: {e}")
        return [], []


def get_all_code_files(root_path='.'):
    """
    Lấy tất cả file code trong dự án
    """
    code_files = []
    exclude_dirs = {'.git', '__pycache__', '.venv', 'venv', 'node_modules', '.pytest_cache'}
    
    # Các extension file code cần phân tích
    code_extensions = {'.py', '.js', '.ts', '.jsx', '.tsx', '.vue', '.php', '.java', '.c', '.cpp', '.cs', '.html', '.css', '.json', '.xml', '.yaml', '.yml', '.md', '.txt', '.sql'}
    
    for root, dirs, files in os.walk(root_path):
        dirs[:] = [d for d in dirs if d not in exclude_dirs]
        
        for file in files:
            file_ext = os.path.splitext(file)[1].lower()
            if file_ext in code_extensions:
                file_path = os.path.join(root, file)
                code_files.append(file_path)
    
    return code_files


def generate_report(output_file='project_analysis.txt'):
    """
    Tạo báo cáo tổng hợp và xuất ra file txt với đầy đủ nội dung code
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("=" * 80 + "\n")
        f.write("              PHÂN TÍCH CẤU TRÚC DỰ ÁN VÀ NỘI DUNG CODE\n")
        f.write("=" * 80 + "\n\n")
        
        # 1. Cấu trúc thư mục
        f.write("1. CẤU TRÚC THƯ MỤC:\n")
        f.write("-" * 40 + "\n")
        
        structure = get_project_structure()
        for line in structure:
            f.write(line + "\n")
        
        f.write("\n")
        
        # 2. Phân tích từng file code với đầy đủ nội dung
        f.write("2. PHÂN TÍCH CÁC FILE CODE VÀ NỘI DUNG:\n")
        f.write("-" * 40 + "\n")
        
        code_files = get_all_code_files()
        
        if not code_files:
            f.write("Không tìm thấy file code nào trong dự án.\n")
        else:
            python_files = [f for f in code_files if f.endswith('.py')]
            js_files = [f for f in code_files if f.endswith(('.js', '.jsx', '.ts', '.tsx'))]
            other_files = [f for f in code_files if f not in python_files and f not in js_files]
            
            # Phân tích file Python với đầy đủ nội dung
            if python_files:
                f.write("\n🐍 PYTHON FILES:\n")
                f.write("=" * 50 + "\n")
                
                for file_path in python_files:
                    f.write(f"\n📁 File: {file_path}\n")
                    f.write("-" * 30 + "\n")
                    
                    # Phân tích cấu trúc
                    classes, functions = analyze_python_file(file_path)
                    
                    if classes:
                        f.write("🏛️  CLASSES:\n")
                        for cls in classes:
                            f.write(f"   • {cls['name']} (dòng {cls['line']})\n")
                            if cls['methods']:
                                for method in cls['methods']:
                                    params_str = ", ".join(method['params'])
                                    f.write(f"     - {method['name']}({params_str}) (dòng {method['line']})\n")
                    
                    if functions:
                        f.write("🔧 FUNCTIONS:\n")
                        for func in functions:
                            params_str = ", ".join(func['params'])
                            f.write(f"   • {func['name']}({params_str}) (dòng {func['line']})\n")
                    
                    # Thêm đầy đủ nội dung file
                    f.write("\n📄 NỘI DUNG FILE:\n")
                    f.write("=" * 30 + "\n")
                    content = get_file_content(file_path)
                    f.write(content)
                    f.write("\n" + "=" * 30 + "\n")
            
            # Phân tích file JavaScript/TypeScript với đầy đủ nội dung
            if js_files:
                f.write("\n🟨 JAVASCRIPT/TYPESCRIPT FILES:\n")
                f.write("=" * 50 + "\n")
                
                for file_path in js_files:
                    f.write(f"\n📁 File: {file_path}\n")
                    f.write("-" * 30 + "\n")
                    
                    # Phân tích cấu trúc
                    functions, classes, exports = analyze_javascript_file(file_path)
                    
                    if classes:
                        f.write("🏛️  CLASSES:\n")
                        for cls in classes:
                            f.write(f"   • {cls['name']} (dòng {cls['line']})\n")
                    
                    if functions:
                        f.write("🔧 FUNCTIONS:\n")
                        for func in functions:
                            f.write(f"   • {func['name']} (dòng {func['line']}) - {func['type']}\n")
                    
                    if exports:
                        f.write("📤 EXPORTS:\n")
                        for exp in exports:
                            f.write(f"   • {exp['name']} (dòng {exp['line']})\n")
                    
                    # Thêm đầy đủ nội dung file
                    f.write("\n📄 NỘI DUNG FILE:\n")
                    f.write("=" * 30 + "\n")
                    content = get_file_content(file_path)
                    f.write(content)
                    f.write("\n" + "=" * 30 + "\n")
            
            # Phân tích các file khác với đầy đủ nội dung
            if other_files:
                f.write("\n📄 OTHER CODE FILES:\n")
                f.write("=" * 50 + "\n")
                
                for file_path in other_files:
                    f.write(f"\n📁 File: {file_path}\n")
                    f.write("-" * 30 + "\n")
                    
                    # Thêm đầy đủ nội dung file
                    f.write("📄 NỘI DUNG FILE:\n")
                    f.write("=" * 30 + "\n")
                    content = get_file_content(file_path)
                    f.write(content)
                    f.write("\n" + "=" * 30 + "\n")
        
        # 3. Thống kê tổng quan
        f.write("\n3. THỐNG KÊ TỔNG QUAN:\n")
        f.write("-" * 40 + "\n")
        
        total_files = len(code_files)
        python_count = len([f for f in code_files if f.endswith('.py')])
        js_count = len([f for f in code_files if f.endswith(('.js', '.jsx', '.ts', '.tsx'))])
        other_count = total_files - python_count - js_count
        
        f.write(f"📊 Tổng số file code: {total_files}\n")
        f.write(f"📊 File Python: {python_count}\n")
        f.write(f"📊 File JavaScript/TypeScript: {js_count}\n")
        f.write(f"📊 File khác: {other_count}\n")
        
        # Tính tổng số dòng code
        total_lines = 0
        for file_path in code_files:
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    lines = len(file.readlines())
                    total_lines += lines
            except:
                pass
        
        f.write(f"📊 Tổng số dòng code: {total_lines}\n")
        
        f.write("\n" + "=" * 80 + "\n")
        f.write("Báo cáo được tạo thành công với đầy đủ nội dung code!\n")
        f.write("=" * 80 + "\n")


# Chạy chương trình
if __name__ == "__main__":
    print("Đang phân tích cấu trúc dự án và lấy nội dung code...")
    generate_report()
    print("✅ Hoàn thành! Kiểm tra file 'project_analysis.txt'")


Đang phân tích cấu trúc dự án và lấy nội dung code...
✅ Hoàn thành! Kiểm tra file 'project_analysis.txt'
